<a href="https://colab.research.google.com/github/sparks-baird/mat_discover/blob/main/examples/structurally-aware-mat-discover-bare-bones.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DiSCoVeR Bare Bones Script

A self-contained, bare-bones example of the DiSCoVeR algorithm.

## Installation

joblib==1.1.0 temporary (no longer needed when hdbscan > 0.8.28).

In [20]:
%pip install hdbscan umap-learn crabnet chem_wasserstein joblib==1.1.0 matbench m3gnet matbench-genmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 9.8 MB/s 
     |████████████████████████████████| 58 kB 5.1 MB/s 
     |████████████████████████████████| 561 kB 20.0 MB/s 
     |████████████████████████████████| 2.7 MB 42.1 MB/s 
     |████████████████████████████████| 256 kB 44.5 MB/s 
     |████████████████████████████████| 12.5 MB 49.2 MB/s 
     |████████████████████████████████| 11.2 MB 48.8 MB/s 
     |████████████████████████████████| 960 kB 53.0 MB/s 
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.2.2
    Uninstalling matplotlib-3.2.2:
      Successfully uninstalled matplotlib-3.2.2


Outline
-----
1. Load some data
2. CrabNet target predictions
3. ElM2D distance calculations
4. DensMAP embeddings and densities
5. Train contribution to validation density
6. Nearest neighbor properties
7. Calculation of weighted scores

In [1]:
dummy = True #@param {type:"boolean"}

## Imports

In [15]:
from operator import attrgetter

import numpy as np
import pandas as pd
import tensorflow as tf

from scipy.stats import multivariate_normal
from scipy.spatial.distance import pdist, squareform
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import RobustScaler

import umap

from m3gnet.models import M3GNet
from m3gnet.trainers import Trainer
from matbench.bench import MatbenchBenchmark
from matbench_genmetrics.utils.featurize import cdvae_cov_struct_fingerprints

## 1. Data

The "index" column helps with knowing where repeat formulas came from. `mat_discover` takes into account when repeat formulas are present in the training dataset (i.e. when calculating the train contribution to log validation density). However, this isn't addressed here; we assume each formula is unique and ignore any repeats.

In [3]:
mb = MatbenchBenchmark(autoload=True, subset=["matbench_log_kvrh"])

2022-10-04 18:58:54 INFO     Initialized benchmark 'matbench_v0.1' with 1 tasks: 
['matbench_log_kvrh']


INFO:matbench:Initialized benchmark 'matbench_v0.1' with 1 tasks: 
['matbench_log_kvrh']


In [25]:
task = list(mb.tasks)[0]
fold = 0
train_inputs, train_outputs = task.get_train_and_val_data(fold)
test_inputs, test_outputs = task.get_test_data(fold, include_target=True)
if dummy:
  train_inputs = train_inputs.head(10)
  train_outputs = train_outputs.head(10)
  test_inputs = test_inputs.head(5)
  test_outputs = test_outputs.head(5)
train_df = pd.concat((train_inputs, train_outputs), axis=1)
val_df = pd.concat((test_inputs, test_outputs), axis=1)
train_df

,structure,log10(K_VRH)
mbid,,
mb-log-kvrh-00001,"[[0. 0. 0.] Ca, [1.37728887 1.57871271 3.73949...",1.707570
mb-log-kvrh-00002,"[[3.14048493 1.09300401 1.64101398] Mg, [0.625...",1.633468
mb-log-kvrh-00003,"[[ 2.06884519 2.40627241 -0.45891585] Si, [1....",1.908485
mb-log-kvrh-00004,"[[2.06428082 0. 2.06428082] Pd, [0. ...",2.117271
mb-log-kvrh-00005,"[[3.09635262 1.0689416 1.53602403] Mg, [0.593...",1.690196
mb-log-kvrh-00007,"[[1.74050602 1.74050602 1.74050602] Pd, [0. 0....",1.995635
mb-log-kvrh-00008,"[[0. 0. 0.] Pd, [0. 0. 2.87508...",1.991226
mb-log-kvrh-00009,"[[0. 0. 1.74916285] Si, [3.211...",2.320146
mb-log-kvrh-00011,"[[0. 2.11129254 2.11129254] Al, [2.111...",1.886491


## 2. MegNet predictions

In [5]:
m3gnet = M3GNet(is_intensive=False)
trainer = Trainer(m3gnet, tf.keras.optimizers.Adam(1e-3))

trainer.train(
    train_inputs.tolist(),
    train_outputs.tolist(),
    fit_per_element_offset=True,
    save_checkpoint=False,
    epochs=2,
)

train_pred = trainer.model.predict_structures(train_inputs)
val_pred = trainer.model.predict_structures(test_inputs)

pred = np.concatenate((train_pred, val_pred), axis=0)

val_rmse = mean_squared_error(test_outputs, val_pred, squared=False)

print("val RMSE: ", val_rmse)


Epoch 1/2
1/1 [==============================] - 8s 8s/step - loss: 1.9631e-04
Epoch 2/2
1/1 [==============================] - 0s 114ms/step - loss: 0.0011
val RMSE:  1.6783232255927492


### Data Preparation

In [6]:
all_structures = pd.concat((train_inputs, test_inputs), axis=0)
all_target = pd.concat((train_outputs, test_outputs), axis=0)
print(all_structures, all_target)

mbid
mb-log-kvrh-00001    [[0. 0. 0.] Ca, [1.37728887 1.57871271 3.73949...
mb-log-kvrh-00002    [[3.14048493 1.09300401 1.64101398] Mg, [0.625...
mb-log-kvrh-00003    [[ 2.06884519  2.40627241 -0.45891585] Si, [1....
mb-log-kvrh-00004    [[2.06428082 0.         2.06428082] Pd, [0.   ...
mb-log-kvrh-00005    [[3.09635262 1.0689416  1.53602403] Mg, [0.593...
mb-log-kvrh-00007    [[1.74050602 1.74050602 1.74050602] Pd, [0. 0....
mb-log-kvrh-00008    [[0. 0. 0.] Pd, [0.         0.         2.87508...
mb-log-kvrh-00009    [[0.         0.         1.74916285] Si, [3.211...
mb-log-kvrh-00011    [[0.         2.11129254 2.11129254] Al, [2.111...
mb-log-kvrh-00012    [[0.       0.       3.549319] N, [0. 0. 0.] N,...
mb-log-kvrh-00006    [[0. 0. 0.] Al, [1.41205261 1.41205261 2.03235...
mb-log-kvrh-00010    [[0.48499983 1.33821071 4.95913808] O, [2.3796...
mb-log-kvrh-00040    [[-1.51512416e-04  5.18156098e-05  6.38120505e...
mb-log-kvrh-00047    [[1.6531378  0.98950681 2.98949123] F, [4.0229...
m

In [7]:
ntrain, nval = len(train_inputs), len(test_inputs)
ntot = ntrain + nval
train_ids, val_ids = np.arange(ntrain), np.arange(ntrain, ntot)
print(train_ids, val_ids)

[0 1 2 3 4 5 6 7 8 9] [10 11 12 13 14]


## 3. Distance calculations

In [8]:
struct_fingerprints = cdvae_cov_struct_fingerprints(all_structures)


/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3441: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


(105,)

In [17]:
dm = squareform(pdist(struct_fingerprints))
dm

array([[0.        , 0.37025253, 0.19258748, 1.06530877, 0.31112306,
        1.24170994, 0.98408477, 0.35741901, 1.07179753, 1.08145982,
        1.15785023, 0.78497011, 1.2712176 , 0.88045732, 1.48451359],
       [0.37025253, 0.        , 0.53102321, 1.06328827, 0.0884027 ,
        1.30941935, 1.04161895, 0.41502179, 1.11413855, 1.12325477,
        1.31709812, 0.74646778, 0.91881587, 0.94413839, 1.50972363],
       [0.19258748, 0.53102321, 0.        , 1.11019835, 0.46818642,
        1.16590386, 0.94867758, 0.44785563, 1.10261709, 1.07326166,
        1.13139612, 0.82315499, 1.4344582 , 0.86120683, 1.48054974],
       [1.06530877, 1.06328827, 1.11019835, 0.        , 1.0618268 ,
        1.75996989, 1.49708071, 1.06154691, 0.21425931, 1.50993465,
        0.86405008, 1.29011082, 1.41253773, 1.41565423, 1.81439484],
       [0.31112306, 0.0884027 , 0.46818642, 1.0618268 , 0.        ,
        1.2912273 , 1.01418538, 0.36585736, 1.10527837, 1.09562808,
        1.28888935, 0.72708563, 0.98691093, 

## 4. DensMAP embeddings and densities

In [18]:
umap_trans = umap.UMAP(
    densmap=True,
    output_dens=True,
    dens_lambda=1.0,
    n_neighbors=30,
    min_dist=0,
    n_components=2,
    metric="precomputed",
    random_state=42,
    low_memory=False,
).fit(dm)


# Extract densMAP embedding and radii
umap_emb, r_orig_log, r_emb_log = attrgetter("embedding_", "rad_orig_", "rad_emb_")(
    umap_trans
)
umap_r_orig = np.exp(r_orig_log)
umap_r_orig

/usr/local/lib/python3.7/dist-packages/umap/umap_.py:1780: UserWarning: using precomputed metric; inverse_transform will be unavailable
  warn("using precomputed metric; inverse_transform will be unavailable")
/usr/local/lib/python3.7/dist-packages/umap/umap_.py:2345: UserWarning: n_neighbors is larger than the dataset size; truncating to X.shape[0] - 1
  "n_neighbors is larger than the dataset size; truncating to "


array([0.67982185, 0.7119372 , 0.7753536 , 1.1853292 , 0.6833535 ,
       1.8067325 , 1.1368273 , 0.64889264, 1.1324492 , 1.2874413 ,
       1.403293  , 0.8394198 , 1.537061  , 1.0953219 , 1.8652128 ],
      dtype=float32)

## 5. Train contribution to validation density

In [26]:
train_emb = umap_emb[:ntrain]
train_r_orig = umap_r_orig[:ntrain]
val_emb = umap_emb[ntrain:]
val_r_orig = umap_r_orig[ntrain:]

train_df["emb"] = list(map(tuple, train_emb))
train_df["r_orig"] = train_r_orig
val_df["emb"] = list(map(tuple, val_emb))
val_df["r_orig"] = val_r_orig


def my_mvn(mu_x, mu_y, r):
    """Calculate multivariate normal at (mu_x, mu_y) with constant radius, r."""
    return multivariate_normal([mu_x, mu_y], [[r, 0], [0, r]])


mvn_list = list(map(my_mvn, train_emb[:, 0], train_emb[:, 1], train_r_orig))
pdf_list = [mvn.pdf(val_emb) for mvn in mvn_list]
val_dens = np.sum(pdf_list, axis=0)
val_log_dens = np.log(val_dens)

val_df["dens"] = val_dens
val_df

,structure,log10(K_VRH),emb,r_orig,dens
mbid,,,,,
mb-log-kvrh-00006,"[[0. 0. 0.] Al, [1.41205261 1.41205261 2.03235...",2.060698,"(15.58591, 0.3872447)",1.403293,0.413671
mb-log-kvrh-00010,"[[0.48499983 1.33821071 4.95913808] O, [2.3796...",1.845098,"(16.9963, 1.3024964)",0.839420,1.088095
mb-log-kvrh-00040,[[-1.51512416e-04 5.18156098e-05 6.38120505e...,1.886491,"(17.412554, 2.2036467)",1.537061,0.410013
mb-log-kvrh-00047,"[[1.6531378 0.98950681 2.98949123] F, [4.0229...",1.857332,"(17.146864, -0.16018976)",1.095322,0.899238
mb-log-kvrh-00049,"[[-0.85832332 4.5043339 2.42315696] Li, [-0...",2.021189,"(16.662127, 2.5661209)",1.865213,0.180679


## 6. Nearest neighbor calculations

In [27]:
r_strength = 1.5
mean, std = (np.mean(dm), np.std(dm))
radius = mean - r_strength * std
n_neighbors = 10
NN = NearestNeighbors(radius=radius, n_neighbors=n_neighbors, metric="precomputed")
NN.fit(dm)

neigh_ind = NN.kneighbors(return_distance=False)
num_neigh = n_neighbors * np.ones(neigh_ind.shape[0])

neigh_target = np.array([pred[ind] for ind in neigh_ind], dtype="object")
k_neigh_avg_targ = np.array(
    [np.mean(t) if len(t) > 0 else float(0) for t in neigh_target]
)

val_k_neigh_avg = k_neigh_avg_targ[val_ids]

## 7. Weighted scores

In [39]:
# deal with warning related to ravel, maybe Colab-specific?
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()

def weighted_score(pred, proxy, pred_weight=1.0, proxy_weight=1.0):
    """Calculate weighted discovery score using the predicted target and proxy."""
    pred = pred.ravel().reshape(-1, 1)
    proxy = proxy.ravel().reshape(-1, 1)
    # Scale and weight the cluster data
    pred_scaler = RobustScaler().fit(pred)
    pred_scaled = pred_weight * pred_scaler.transform(pred)
    proxy_scaler = RobustScaler().fit(-1 * proxy)
    proxy_scaled = proxy_weight * proxy_scaler.transform(-1 * proxy)

    # combined cluster data
    comb_data = pred_scaled + proxy_scaled
    comb_scaler = RobustScaler().fit(comb_data)

    # cluster scores range between 0 and 1
    score = comb_scaler.transform(comb_data).ravel()
    return score


peak_score = weighted_score(val_pred, val_k_neigh_avg)
dens_score = weighted_score(val_pred, val_dens)
pd.DataFrame(dict(dens=dens_score, peak=peak_score)).sort_values(by="dens", ascending=False)

,dens,peak
3,1.935662,4.116384
4,0.914495,0.622159
2,0.000000,-0.377841
0,-0.085505,-0.440964
1,-1.021639,0.000000
